In [ ]:
import snap
import re
from collections import defaultdict

In [ ]:
post_file = 'output/reddit_submissions_jan2012.txt'
comment_file = 'output/reddit_comments_jan2012.txt'

In [ ]:
int_attrs = ['score', 'gilded', 'created_utc']
str_attrs = ['author', 'text', 'id']
def make_net():
    net = snap.TNEANet.New()

    for ia in int_attrs:
        net.AddIntAttrN(ia)
    for sa in str_attrs:
        net.AddStrAttrN(sa)
    
    return net

In [ ]:
posts = open(post_file)
header = posts.readline()[1:].rstrip('\n').split('\t')
fields = {name: i for (i, name) in enumerate(header)}

nets = {} # subreddit name to a TNEANet (e.g. 'politics' -> tneanet)

# Parse post info and add to table
for (i, post) in enumerate(posts):
    entries = post.rstrip('\n').split('\t')
    subreddit = entries[fields['Subreddit']]
    if subreddit not in nets:
        nets[subreddit] = make_net()
    srnet = nets[subreddit]
    
    post_id = entries[fields['Post_ID']]
    author = entries[fields['Author']]
    title = entries[fields['Title']]
    score = int(entries[fields['Score']])
    gilded = int(entries[fields['Gilded']])
    created_utc = int(entries[fields['Created_UTC']])
    
    nid = srnet.AddNode()
    postids_to_nids[post_id] = nid
    
    srnet.AddIntAttrDatN(nid, score, 'score')
    srnet.AddIntAttrDatN(nid, gilded, 'gilded')
    srnet.AddIntAttrDatN(nid, created_utc, 'created_utc')
    srnet.AddStrAttrDatN(nid, author, 'author')
    srnet.AddStrAttrDatN(nid, title, 'text')
    srnet.AddStrAttrDatN(nid, post_id, 'id')
    
    if i % 100000 == 0:
        print(i)

posts.close()

In [ ]:
comments = open(comment_file)
header = comments.readline()[1:].rstrip('\n').split('\t')
fields = {name: i for (i, name) in enumerate(header)}

# Reduce memory pressure by reading table once for each subreddit
srnames_to_lines = defaultdict(set)
for (i, com) in enumerate(comments):
    entries = com.rstrip('\n').split('\t')
    subreddit = entries[fields['Subreddit']]
    srnames_to_lines[subreddit].add(i)

comments.close()

In [ ]:
output_directory = 'output/subreddit_nets'
os.makedirs(output_directory)

# This code will iterate over one subreddit at a time to make one TNEANet per subreddit. 
# This takes a long time but saves a lot of memory; iterating over the file once to parse all subreddits uses
# several times more memory and is difficult to do on local machines.
names = (name for name in sorted(srnames_to_lines.keys()))
for subreddit in names:
    print(subreddit)
    srlines = srnames_to_lines[subreddit]
    srnet = nets[subreddit]
    
    comments = open(comment_file)
    header = comments.readline()[1:].rstrip('\n').split('\t')
    fields = {name: i for (i, name) in enumerate(header)}
    comids_to_nids = {}

    comlines = (line for (j, line) in enumerate(comments) if j in srlines)
    for (i, com) in enumerate(comlines):
        entries = com.rstrip('\n').split('\t')
        is_reply = bool(int(entries[fields['Is_Reply']]))
        parent_id = entries[fields['Parent_ID']]
        comment_id = entries[fields['Comment_ID']]

        if subreddit in nets and parent_id in postids_to_nids or parent_id in comids_to_nids: 
            # Comment not orphaned; add to graph
            author = entries[fields['Commenter']]
            text = entries[fields['Comment_Text']]
            score = int(entries[fields['Score']])
            gilded = int(entries[fields['Gilded']])
            created_utc = int(entries[fields['Created_UTC']])

            nid = srnet.AddNode()
            comids_to_nids[comment_id] = nid
            parent_nid = (comids_to_nids[parent_id] if is_reply else postids_to_nids[parent_id])
            srnet.AddEdge(nid, parent_nid)

            srnet.AddIntAttrDatN(nid, score, 'score')
            srnet.AddIntAttrDatN(nid, gilded, 'gilded')
            srnet.AddIntAttrDatN(nid, created_utc, 'created_utc')
            srnet.AddStrAttrDatN(nid, author, 'author')
            srnet.AddStrAttrDatN(nid, text, 'text')
            srnet.AddStrAttrDatN(nid, comment_id, 'id')
    
    out = snap.TFOut(output_directory + '/' + subreddit + '.graph')
    srnet.Save(out)
    out.Flush()
    srnet.Clr()
    
    del srnames_to_lines[subreddit]
    del nets[subreddit]
    comments.close()